## 1. Carga de librerias


In [ ]:
! python -m pip install --upgrade pip
! pip install keras
! pip install tensorflow

In [2]:
### General
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
import math
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score
import tensorflow as tf

### Error
## ECM
from sklearn.metrics import mean_squared_error
## Coeficiente de determinacion
from sklearn.metrics import r2_score
## Varianza explicada
from sklearn.metrics import explained_variance_score
## Naive Bayes 
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
## KNN
from sklearn.neighbors import KNeighborsClassifier
### Redes Neuronales Artificiales
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
### Random Forest
from sklearn.ensemble import RandomForestClassifier
### Vectores de soporte
from sklearn import svm

Using TensorFlow backend.


## 2. Carga de la base de datos

#### Características: 
- Variables de entrada:
 - ##### Datos del cliente del banco:
   1. age (numérica)
   2. job : tipo de empleo (categórica: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",     "blue-collar","self-employed","retired","technician","services") 
   3. marital : estado civil (categórica: "married","divorced","single"; nota: "divorced" aplica para divorciado o viudo)
   4. education (categórica: "unknown","secondary","primary","tertiary")
   5. default: tiene credito? (binaria: "yes","no")
   6. balance: promedio de saldo anual en euros (numérica) 
   7. housing: tiene crédito de vivienda? (binaria: "yes","no")
   8. loan: tiene crédito personal? (binaria: "yes","no")
 - ##### Datos del último contacto en relación a la actual campaña:
   9. contact: tipo de comunicación (categórica: "unknown","telephone","cellular") 
   10. day: último día de contacto en el mes (numérica)
   11. month: último mes de contacto del año (categórica: "jan", "feb", "mar", ..., "nov", "dec")
   12. duration: última duración de contacto en segundos (numérica)
 - ##### Otros atributos:
   13. campaign: número de contactos realizados durante la campaña para este cliente (numérica, incluye último contacto)
   14. pdays: número de dias que pasaron después de que el cliente fué contactado desde una campaña anterior (numérica, -1 significa que el cliente no había sido previamente contactado)
   15. previous: número de contactos realizados antes de la campaña actual para este cliente (numérica)
   16. poutcome: resultado de la campaña anterior (categórica: "unknown","other","failure","success")
   
   
- Variable de salida (objetivo deseado):

   17. y: se suscribió el cliente al depósito a término fijo? (binaria: "yes","no")


- Datos faltantes: nunguno

In [3]:
database = pd.read_csv("data/bank.csv", delimiter=";",header='infer')

#job = ['admin.', 'unknown', 'unemployed', 'management', 'housemaid', 'entrepreneur', 'student', 
#       'blue-collar', 'self-employed', 'retired', 'technician', 'services']
#marital = ['married', 'divorced', 'single']
#education =['unknown', 'secondary', 'primary', 'tertiary']
#default = ['yes', 'no']
#housing = ['yes', 'no']
#loan = ['yes', 'no']
#contact = ['unknown', 'telephone', 'cellular']
#month = ['jan', 'feb', 'mar', 'apr', 'jun', 'jul', 'aug', 'sep', 'oct' 'nov', 'dec']
#poutcome = ['unknown', 'other', 'failure', 'success']

data_encoded = pd.get_dummies(database, columns=['job','marital',
                                         'education','default',
                                         'housing','loan',
                                         'contact','month',
                                         'poutcome'])

# Convertir salida a binario
data_encoded.y.replace(('yes', 'no'), (1, 0), inplace=True)

data = data_encoded.iloc[:,0:51]
target = data_encoded.iloc[:,-1]

print(data_encoded.shape)
print(data.shape)
print(target.shape)

data_yes = data_encoded.loc[data_encoded['y'] == 1]
data_no = data_encoded.loc[data_encoded['y'] == 0]

print('Datos con salida yes: ', data_yes.shape)
print('Datos con salida no:', data_no.shape)

# Dividir en conjuntos de entrenamiento y pruebas
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3) # 70% training and 30% test


(4521, 52)
(4521, 51)
(4521,)
Datos con salida yes:  (521, 52)
Datos con salida no: (4000, 52)


## Naive Bayes

In [4]:
## Naive Bayes 
gnb = GaussianNB()
bnb = BernoulliNB()

gnb_pred = gnb.fit(X_train, y_train).predict(X_test)

print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
      .format(
          X_test.shape[0],
          (y_test != gnb_pred).sum(),
          100*(1-(y_test != gnb_pred).sum()/X_test.shape[0])
))

Number of mislabeled points out of a total 1357 points : 3, performance 99.78%


## KNN

In [5]:
knn = KNeighborsClassifier(n_neighbors=2)

knn.fit(X_train, y_train)

knn_pred = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, knn_pred))

Accuracy: 0.9683124539425203


## RNA

In [6]:
model = Sequential()

# Add an input layer 
model.add(Dense(64, activation='relu', input_shape=(51,)))

# Add an output layer 
model.add(Dense(1, activation='sigmoid'))

# For a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          validation_data=(X_test, y_test),
          epochs=20,
          verbose=2)
score = model.evaluate(X_test, y_test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 3164 samples, validate on 1357 samples
Epoch 1/20
 - 0s - loss: 1.1434 - acc: 0.9042 - val_loss: 0.7072 - val_acc: 0.9484
Epoch 2/20
 - 0s - loss: 0.4073 - acc: 0.9655 - val_loss: 0.1494 - val_acc: 0.9867
Epoch 3/20
 - 0s - loss: 0.4265 - acc: 0.9589 - val_loss: 0.0326 - val_acc: 0.9948
Epoch 4/20
 - 0s - loss: 0.1885 - acc: 0.9820 - val_loss: 0.0433 - val_acc: 0.9934
Epoch 5/20
 - 0s - loss: 0.2174 - acc: 0.9829 - val_loss: 0.0314 - val_acc: 0.9941
Epoch 6/20
 - 0s - loss: 0.1534 - acc: 0.9861 - val_loss: 0.2840 - val_acc: 0.9786
Epoch 7/20
 - 0s - loss: 0.1714 - acc: 0.9848 - val_loss: 2.6373 - val_acc: 0.7539
Epoch 8/20
 - 0s - loss: 0.2351 - acc: 0.9798 - val_loss: 0.0499 - val_acc: 0.9941
Epoch 9/20
 - 0s - loss: 0.1665 - acc: 0.9864 - val_loss: 0.0185 - val_acc: 0.9971
Epoch 10/20
 - 0s - loss: 0.1659 - acc: 0.9845 - val_loss: 0.0199 - val_acc: 0.9971
E

In [7]:
y_pred = model.predict(X_test)
print(y_pred)
print('Score: ', score)

# Confusion matrix
confusion_matrix(y_test, y_pred)

# Precision 
precision_score(y_test, y_pred)

# Recall
recall_score(y_test, y_pred)

# F1 score
f1_score(y_test,y_pred)

# Cohen's kappa
cohen_kappa_score(y_test, y_pred)

[[1.]
 [0.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
Score:  [0.03368849134601004, 0.9963154016212233]


ValueError: Classification metrics can't handle a mix of binary and continuous targets

## Random Forest

In [14]:
rfc = RandomForestClassifier(n_estimators=5)

#train model
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

print('Score: ', rfc.score(X_train, y_train))

print("Accuracy:", accuracy_score(y_test, rfc_pred))

rfc_results = cross_validate(rfc, X_train, y_train, cv=5)
print(np.mean(rfc_results['test_score']), np.mean(rfc_results['train_score']))

Score:  1.0
Accuracy: 1.0
1.0 1.0


/home/daemonsoft/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


## Maquinas de Soporte Vectorial con kernel lineal y con kernel RBF

In [ ]:
# Lineal







In [ ]:
#RBF


